In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
IN = Path('/Users/rushil/brain_extraction/results/quantitative/rim_metrics_r1_scan.csv')
OUTDIR = Path('/Users/rushil/brain_extraction/results/quantitative/figures')
OUTDIR.mkdir(parents=True, exist_ok=True)

df = pd.read_csv(IN)
methods = df['method'].unique().tolist()

exclude = ['6109-317_20150302_0647_ct', '6142-308_20150610_0707_ct', '6193-324_20150924_1431_ct', '6257-335_20160118_1150_ct',
                     '6418-193_20161228_1248_ct', '6470-296_20170602_0607_ct', '6480-154_20170622_0937_ct']

exclude_prefixes = ("6046", "6084", "6096", "6246", "6315", "6342", "6499")

df = df[~df["stem"].isin(exclude)]
df = df[~df["stem"].str.startswith(exclude_prefixes)]

print(len(df) == 6*5062 + 2403)

df['method'].value_counts()

True


method
SynthStrip      5062
Robust-CTBET    5062
Brainchop       5062
HD-CTBET        5062
CTbet_Docker    5062
CTBET           5062
CT_BET          2403
Name: count, dtype: int64

In [9]:
# thresholds to evaluate
thresh = {
	'frac_bone':[0.1,0.25,0.5],
	'vol_bone_ml':[50,100],
	'frac_air':[0.001,0.005],
	'vol_air_ml':[0.1,0.5,1],
	'p95':[1000],
}

# Subject-level dataframe (used for subject-wise per-method table)
subj = pd.read_csv('/Users/rushil/brain_extraction/results/quantitative/rim_metrics_r1_subj.csv')

# Plots: method-wise p95 CDF
sns.set(style='whitegrid')
plt.figure(figsize=(8,6))
for method in methods:
	s = df.loc[df['method']==method,'p95'].dropna()
	if s.size==0: continue
	vals = np.sort(s.values)
	cdf = np.arange(1, len(vals)+1)/len(vals)
	plt.step(vals, cdf, where='post', label=method)
plt.xscale('symlog')
plt.xlabel('p95 HU (log scale)')
plt.ylabel('CDF')
plt.legend()
plt.title('Per-method CDF of rim p95')
plt.tight_layout()
plt.savefig(OUTDIR / 'p95_cdf_per_method.png', dpi=150)
plt.close()
print('Wrote p95 CDF to', OUTDIR / 'p95_cdf_per_method.png')

# --- per-method percent-above-threshold table (methods as columns) for scans ---
row_keys = []
for metric, ts in thresh.items():
	for t in ts:
		key = f"{metric}_{t}"
		row_keys.append(key)

method_table = {m: [] for m in methods}
for metric, ts in thresh.items():
	for t in ts:
		for m in methods:
			sub = df[df['method'] == m]
			if len(sub) == 0:
				pct = 0.0
			else:
				pct = float((sub[metric].fillna(0) > t).sum()) / float(len(sub)) * 100.0
			method_table[m].append(pct)

method_pct_df = pd.DataFrame(method_table, index=row_keys)
method_pct_df.index.name = 'metric_threshold'
method_pct_df.to_csv(OUTDIR / 'method_pct_above_thresholds.csv')
print('Wrote per-method percent-above-thresholds (scans) to', OUTDIR / 'method_pct_above_thresholds.csv')

# --- per-method percent-above-threshold table (methods as columns) for subjects ---
method_table_subj = {m: [] for m in methods}
for metric, ts in thresh.items():
	for t in ts:
		for m in methods:
			subs = subj[subj['method'] == m]
			if len(subs) == 0:
				pct = 0.0
			else:
				pct = float((subs[metric].fillna(0) > t).sum()) / float(len(subs)) * 100.0
			method_table_subj[m].append(pct)

method_pct_df_subj = pd.DataFrame(method_table_subj, index=row_keys)
method_pct_df_subj.index.name = 'metric_threshold'
method_pct_df_subj.to_csv(OUTDIR / 'method_pct_above_thresholds_subjects.csv')
print('Wrote per-method percent-above-thresholds (subjects) to', OUTDIR / 'method_pct_above_thresholds_subjects.csv')

# p99 CDF plot (per-method)
sns.set(style='whitegrid')
plt.figure(figsize=(8,6))
for method in methods:
    s = df.loc[df['method']==method,'p99'].dropna()
    if s.size == 0:
        continue
    vals = np.sort(s.values)
    cdf = np.arange(1, len(vals)+1) / len(vals)
    plt.step(vals, cdf, where='post', label=method)
plt.xscale('symlog')
plt.xlabel('p99 HU (log scale)')
plt.ylabel('CDF')
plt.legend()
plt.title('Per-method CDF of rim p99')
plt.tight_layout()
plt.savefig(OUTDIR / 'p99_cdf_per_method.png', dpi=150)
plt.close()
print('Wrote p99 CDF to', OUTDIR / 'p99_cdf_per_method.png')


Wrote p95 CDF to /Users/rushil/brain_extraction/results/quantitative/figures/p95_cdf_per_method.png
Wrote per-method percent-above-thresholds (scans) to /Users/rushil/brain_extraction/results/quantitative/figures/method_pct_above_thresholds.csv
Wrote per-method percent-above-thresholds (subjects) to /Users/rushil/brain_extraction/results/quantitative/figures/method_pct_above_thresholds_subjects.csv
Wrote p99 CDF to /Users/rushil/brain_extraction/results/quantitative/figures/p99_cdf_per_method.png
Wrote p99 CDF to /Users/rushil/brain_extraction/results/quantitative/figures/p99_cdf_per_method.png
